In [ ]:
#import data from a csv file and store in a dataframe
import pandas as pd
df = pd.read_csv('../data/Hotel_Reviews.csv')



df['Negative_Review'].isnull().sum(), df['Positive_Review'].isnull().sum()
# Clean the reviews from the dataset
#removing invalid ones, e.g. empty reviews, everything is good/bad and reviews with less than 10 characters
# output the df with two new columns: 'Negative_clean' and 'Positive_clean'

# process the Negative_Review column
# strip() to remove leading and trailing whitespaces for one column of df
import numpy as np

#change data type to string for the column NegativeReview
df['NegativeReview'] = df['Negative_Review'].astype(str)

#show data type for the column NegativeReview
print(type(df['NegativeReview'][0]))

df['Negative_clean'] = df['NegativeReview'].apply(lambda x: x.strip())

# if the text has less than 10 characters, replace the text with 'nothing' for the above column
df.loc[df['Negative_clean'].str.len() < 10, 'Negative_clean'] = ' '

# if the text has 'nothing' or 'everything' or 'anything' str in it regardless lower case, replace the text with 'nothing' for the above column
df.loc[df['Negative_clean'].str.contains('nothing|everything|anything|No Negative', case=False), 'Negative_clean'] = ' '



# if the text has 'nothing' or 'everything' or 'No Positive' str in it regardless lower case, replace the text with 'nothing' for the above column
df.loc[df['Positive_clean'].str.contains('nothing|everything|No Positive', case=False), 'Positive_clean'] = ' '
df['Positive_clean'].value_counts().head(30)


In [ ]:
# check ' ' value for the new columns
neg_invalid_count = df['Negative_clean'][df['Negative_clean']==" "].shape[0]
pos_invalid_count = df['Positive_clean'][df['Positive_clean']==" "].shape[0]
pos_invalid_count/len(df), neg_invalid_count/len(df)

In [ ]:
# merger two 'Negative_clean', 'Positive_clean' columns and save to a new column 'Reviews', and save to a new column 'Reviews_clean'
df['Reviews_clean'] = df['Negative_clean'] + df['Positive_clean']
df['Reviews_clean']

In [ ]:
df['Reviews_clean'].value_counts().head(10)



In [ ]:
# if the value is ' ', drop the row
df_new = df[df['Reviews_clean'] != '  '][['Reviews_clean']]
df_new.shape

In [ ]:
df_new.value_counts().head(30)

In [ ]:
type(df_new)
# the data is cleaned and ready for further analysis

In [ ]:
# get the length of each string in column 'Reviews_clean'
df_new['length'] = df_new['Reviews_clean'].str.len()

df_new

In [ ]:
#get the mean of the length, but should check the word count instead, can do it 
#by using nltk library
df_new['length'].mean(), df_new['length'].median(), df_new['length'].max(), df_new['length'].min()

In [ ]:
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer

# function to get the keywords from the text using KeyBERT
def keywords_extract(text):
    sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
    kw_model = KeyBERT(model=sentence_model)
    keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words='english', top_n=100)
    return keywords

doc = df_new['Reviews_clean'][1]


keywords = keywords_extract(doc)
keywords

In [ ]:
test_df = df_new.head(2) 

In [ ]:
# get the list of keywords for each the review in the dataframe, append to a big list

test_df['keywords'] = test_df.apply(lambda x: keywords_extract(x))


In [ ]:
test_df

In [ ]:
# flatten column 'keywords' to get a list of keywords
list_list = test_df['keywords'].tolist()

In [ ]:
# flatten the list of list
one_list = [item for sublist in list_list for item in sublist]
one_list


In [ ]:
# convert the list to a DataFrame
keyword_df = pd.DataFrame(one_list, columns=['keyword', 'score'])
keyword_df

In [ ]:
# get the top 10 keywords by groupby keyword and get the sum of the score
keyword_df.groupby('keyword').sum().sort_values(by='score', ascending=False).head(10)


